In [1]:
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
font_path = 'C:/Windows/Fonts/NGULIM.TTF'
font_family = fm.FontProperties(fname=font_path).get_name()
plt.rcParams["font.family"] = font_family

# 저에게 던져주신 다음 문제 전에 받은 인구 데이터를 이용하여 인구 대비 치킨집 비율 추가하기

In [2]:
res = pd.read_excel('data/6270000_대구광역시_07_24_04_P_일반음식점.xlsx')
res.head()

,번호,개방서비스명,개방서비스ID,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,1,일반음식점,07_24_04_P,3410000,3410000-101-2002-00042,20020321,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,NaN,N,144.58,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,3410000,3410000-101-2002-00043,20020322,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,NaN,N,59.55,NaN,NaN,NaN
2,3,일반음식점,07_24_04_P,3410000,3410000-101-2002-00046,20020322,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,NaN,N,42.78,NaN,NaN,NaN
3,4,일반음식점,07_24_04_P,3410000,3410000-101-2002-00067,20020325,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,NaN,N,77.01,NaN,NaN,NaN
4,5,일반음식점,07_24_04_P,3410000,3410000-101-2002-00041,20020325,NaN,3,폐업,2,...,NaN,NaN,NaN,NaN,NaN,N,140.58,NaN,NaN,NaN


In [3]:
res2 = res[['상세영업상태코드','소재지전체주소','업태구분명']]
res2 = res2.dropna(axis=0)
res2 = res2[res2['상세영업상태코드']==1]
res2 = res2[res2['업태구분명'].isin(['호프/통닭','통닭(치킨)'])]
res2.head()

,상세영업상태코드,소재지전체주소,업태구분명
4589,1,대구광역시 중구 공평동 0058-0006번지,호프/통닭
4604,1,대구광역시 중구 남산동 2120-0001번지,호프/통닭
4627,1,대구광역시 중구 남산동 2380-0023번지,호프/통닭
4684,1,대구광역시 중구 삼덕동1가 0028-0007번지 지상3층,호프/통닭
4699,1,대구광역시 중구 남성로 0094-0001번지,호프/통닭


In [4]:
def f(group):
    return group[2]
res2['동네'] = list(map(f,res2['소재지전체주소'].str.split(' ')))

In [5]:
def f2(group):
    return group[1]
res2['구'] = list(map(f2,res2['소재지전체주소'].str.split(' ')))

In [6]:
res3 = res2[['동네','구']]
res3.columns = ['동네','개수']
res3 = res3.groupby('동네')
res3 = res3.count()
res3.sort_values('개수',ascending=False)

,개수
동네,
대명동,149
다사읍,118
상인동,111
태전동,92
평리동,91
...,...
월암동,1
대림동,1
이현동,1


In [7]:
#똑같이 추출
res4 = res2[['구','동네']]
res4.columns=['구','개수']
res4 = res4.groupby('구')
res4 = res4.count()
res4.sort_values('개수',ascending=False)

,개수
구,
달서구,841
북구,618
수성구,453
동구,422
달성군,394
서구,249
남구,207
중구,183


In [8]:
pop1906 = pd.read_csv('data/2019.06.30.UTF8.csv',thousands=',')
pop1906.rename(columns={'행정구역':'구분'},inplace=True)

In [9]:
#남 녀가 중요하지 않으니 계로 추출 복습
df = pop1906.copy()
drop_list = []  
check_list = ['총계', '구분', ' 남', ' 여'] 
for chk in check_list:  
    for item in df.columns :  
        if  chk in item:  
            drop_list.append(item)  
drop_list = list(set(drop_list))
df3 = df.drop(drop_list,axis=1)
df3['인구'] = (pd.DataFrame(range(0,101))/10).astype(int)*10
df3['인구'] = np.where(df3['인구'] >= 70, 70, df3.인구)
df3 = df3.groupby('인구')
df3 = df3.sum()
df3.columns = df3.columns.str.replace(' 계', '')
df3

,중구,동구,서구,남구,북구,수성구,달서구,달성군
인구,,,,,,,,
0,5572,27937,8765,7221,36116,31376,44236,30293
10,5370,28629,12665,10462,46834,55090,60556,23621
20,11620,42833,23711,20327,60691,56987,81862,29765
30,11311,47415,19635,17871,56384,45226,71171,43412
40,11108,52960,25199,21400,75095,76047,94565,41490
50,11798,58640,36882,26010,76729,74962,105716,40356
60,10069,48272,29378,22886,48297,48484,66056,26081
70,11087,42139,23685,22505,38947,42309,47141,19221


In [10]:
df3=df3.T

In [11]:
df3['총합']=df3.sum(axis='columns').astype(int)

In [12]:
#총합만 추출
df4 = df3['총합']

In [13]:
df4

중구      77935
동구     348825
서구     179920
남구     148682
북구     439093
수성구    430481
달서구    571303
달성군    254239
Name: 총합, dtype: int32

In [14]:
#통닭집에 이어붙이기
res4['인구']=df4

In [15]:
res4

,개수,인구
구,,
남구,207,148682
달서구,841,571303
달성군,394,254239
동구,422,348825
북구,618,439093
서구,249,179920
수성구,453,430481
중구,183,77935


In [16]:
#천명당 치킨집 수
res4['비율']=res4['개수']/(res4['인구']//1000)
res4

,개수,인구,비율
구,,,
남구,207,148682,1.398649
달서구,841,571303,1.472855
달성군,394,254239,1.551181
동구,422,348825,1.212644
북구,618,439093,1.407745
서구,249,179920,1.391061
수성구,453,430481,1.053488
중구,183,77935,2.376623
